In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [29]:
include("utils.jl")
include("visual_utils.jl")

show_eof_and_pc_modes_of_timeline (generic function with 1 method)

In [3]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [4]:
# mon_mean_base_path = "/home/denis/workspace/data/ivt_monthly_mean"
# day_mean_base_path = "/home/denis/workspace/data/ivt_daily_mean"

In [5]:
mon_mean_base_path = "/mnt/bigdrive/Datasets/ivt_monthly_mean"
day_mean_base_path = "/mnt/bigdrive/Datasets/ivt_daily_mean"
full_dataset_path = "/mnt/bigdrive/Datasets/ivt_fields_v1"
ps_data_path = "/mnt/bigdrive/Datasets/ps_data"
ps_data_monthly = "/mnt/bigdrive/Datasets/ps_data_monthly_mean"

"/mnt/bigdrive/Datasets/ps_data_monthly_mean"

In [6]:
# monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection=:)) do timedata
#     return year(timedata) >= 2081 && year(timedata) <= 2100  
# end

In [7]:
winter_months = [12, 1, 2, 3]

4-element Vector{Int64}:
 12
  1
  2
  3

In [8]:
ps_data = filter_by_date(build_timeline_data(ps_data_monthly, 10, "ssp585"; file_range_selection = :, data_field_id="ps")) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [9]:

monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 20, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [49]:
daily_tl_data = filter_by_date(build_timeline_data(day_mean_base_path, 20, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T12:00:00"), DateTime("2015-01-02T09:00:00"), DateTime("2015-01-03T09:00:00"), DateTime("2015-01-04T09:00:00"), DateTime("2015-01-05T09:00:00"), DateTime("2015-01-06T09:00:00"), DateTime("2015-01-07T09:00:00"), DateTime("2015-01-08T09:00:00"), DateTime("2015-01-09T09:00:00"), DateTime("2015-01-10T09:00:00")  …  DateTime("2100-12-23T09:00:

In [ ]:
# truth_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection=1:2)) do timedata
#     return year(timedata) >= 2015 && year(timedata) <= 2034
# end

In [ ]:
compare_truth_with_tldata(truth_data, tl_data, "mon_mean_comparison.mp4"; framerate = 3, resolution = (2000, 1000), colormap = :managua)

In [54]:
scopes_30_seasons = get_sliding_time_scopes_by_threshold(monthly_tl_data.time, 30)

57-element Vector{UnitRange{Int64}}:
 1:120
 4:124
 8:128
 12:132
 16:136
 20:140
 24:144
 28:148
 32:152
 36:156
 ⋮
 194:314
 198:318
 202:322
 206:327
 210:331
 214:335
 218:339
 222:343
 226:347

In [12]:
scopes_50_seasons = get_sliding_time_scopes_by_threshold(monthly_tl_data.time, 50)

37-element Vector{UnitRange{Int64}}:
 1:201
 4:205
 8:209
 12:213
 16:217
 20:221
 24:225
 28:229
 32:233
 36:237
 ⋮
 113:314
 117:318
 121:322
 125:327
 129:331
 133:335
 137:339
 141:343
 145:347

In [13]:
eof_data = calculate_eofs_of_tl_data(
    monthly_tl_data, 
    scopes_50_seasons, 
    2; 
    engine=:python, 
    reof=false, 
    center=true, 
    align_eof_with_mean=true, 
    align_pcs_with_mean=false, 
    weights=sqrt.(cos.(deg2rad.(monthly_tl_data.lats))), 
    eof_type=:normal,
    scale_with_eigenvals=true
)

Handled scope 1:201 out of 37 on thread 1
Handled scope 4:205 out of 37 on thread 1
Handled scope 8:209 out of 37 on thread 1
Handled scope 12:213 out of 37 on thread 1
Handled scope 16:217 out of 37 on thread 1
Handled scope 20:221 out of 37 on thread 1
Handled scope 24:225 out of 37 on thread 1
Handled scope 28:229 out of 37 on thread 1
Handled scope 32:233 out of 37 on thread 1
Handled scope 36:237 out of 37 on thread 1
Handled scope 40:241 out of 37 on thread 1
Handled scope 44:246 out of 37 on thread 1
Handled scope 48:250 out of 37 on thread 1
Handled scope 52:254 out of 37 on thread 1
Handled scope 56:258 out of 37 on thread 1
Handled scope 60:262 out of 37 on thread 1
Handled scope 64:266 out of 37 on thread 1
Handled scope 68:270 out of 37 on thread 1
Handled scope 72:274 out of 37 on thread 1
Handled scope 76:278 out of 37 on thread 1
Handled scope 80:282 out of 37 on thread 1
Handled scope 85:286 out of 37 on thread 1
Handled scope 89:290 out of 37 on thread 1
Handled scope 

Dict{String, Vector{EOFResult}} with 2 entries:
  "ssp126" => [EOFResult([-20.9187 78.0564 … 20.3053 14.7421; -7.27791 124.338 …
  "ssp585" => [EOFResult([-116.306 -13.3401 … 12.0226 9.54506; -96.7593 15.3366…

In [18]:
eof_data_ps_nao = calculate_eofs_of_tl_data(
    ps_data, 
    scopes_50_seasons, 
    1; 
    engine=:python, 
    reof=false, 
    center=true, 
    align_eof_with_mean=true, 
    align_pcs_with_mean=false, 
    weights=sqrt.(cos.(deg2rad.(monthly_tl_data.lats))), 
    eof_type=:normal,
    scale_with_eigenvals=true
)

Handled scope 1:201 out of 37 on thread 1
Handled scope 4:205 out of 37 on thread 1
Handled scope 8:209 out of 37 on thread 1
Handled scope 12:213 out of 37 on thread 1
Handled scope 16:217 out of 37 on thread 1
Handled scope 20:221 out of 37 on thread 1
Handled scope 24:225 out of 37 on thread 1
Handled scope 28:229 out of 37 on thread 1
Handled scope 32:233 out of 37 on thread 1
Handled scope 36:237 out of 37 on thread 1
Handled scope 40:241 out of 37 on thread 1
Handled scope 44:246 out of 37 on thread 1
Handled scope 48:250 out of 37 on thread 1
Handled scope 52:254 out of 37 on thread 1
Handled scope 56:258 out of 37 on thread 1
Handled scope 60:262 out of 37 on thread 1
Handled scope 64:266 out of 37 on thread 1
Handled scope 68:270 out of 37 on thread 1
Handled scope 72:274 out of 37 on thread 1
Handled scope 76:278 out of 37 on thread 1
Handled scope 80:282 out of 37 on thread 1
Handled scope 85:286 out of 37 on thread 1
Handled scope 89:290 out of 37 on thread 1
Handled scope 

Dict{String, Vector{EOFResult}} with 1 entry:
  "ssp585" => [EOFResult([863.202 984.147 … -2806.43 -2596.98; 789.093 927.23 ……

In [30]:
show_eof_modes_of_timeline(
    monthly_tl_data,
    eof_data,
    all_scopes,
    "eof_6modes_monthly_30seasons_aligned_tst.mp4";
    framerate = 2,
    colormap = :broc,
    coastline_color = :black,
    resolution = (2000, 1100),
    whole_figure_title = "Leading EOF Modes of Daily Averages of IVT (18 Winter Period) Aligned"
)

Threads available: 8


Temporal plot first 2 modes
Compare to mean
scale with singular value (diss)
0 contour plot

In [30]:
figure = show_eof_and_pc_modes_of_timeline(
    monthly_tl_data,
    eof_data,
    scopes_50_seasons,
    "eof_ivt_2modes_pc_dailymean_50seasons_naoindex.mp4";
    framerate = 2,
    colormap = :BrBG_3,
    coastline_color = :black,
    resolution = (2000, 2000),
    whole_figure_title = "Leading EOF Modes of Daily Averages of IVT (50 Winter Period)",
    shading=NoShading,
    surface_alpha_value=1.0,
    fontsize=14,
    additional_eof_data=eof_data_ps_nao,
    additional_data_label="NAO index"
)

Data extremas: -582.9081377970413 1030.2213328965045


In [19]:
figure = show_eof_and_pc_modes_of_timeline(
    monthly_tl_data,
    eof_data,
    scopes_30_seasons,
    "eof_ivt_2modes_pc_monthlymean_30seasons_naoindex.mp4";
    framerate = 2,
    colormap = :BrBG_3,
    coastline_color = :black,
    resolution = (2000, 2000),
    whole_figure_title = "Leading EOF Modes of Monthly Averages of IVT (30 Winter Period) with NAO index",
    shading=NoShading,
    surface_alpha_value=1.0,
    fontsize=14,
    additional_eof_data=eof_data_ps_nao,
    additional_data_label="NAO index"
)

Data extremas: -582.9081377970413 1030.2213328965045


In [21]:
figure = show_eof_and_pc_modes_of_timeline(
    ps_data,
    eof_data,
    all_scopes,
    "ps_2modes_pc_mean_monthly_50seasons.mp4";
    framerate = 2,
    colormap = :BrBG_3,
    coastline_color = :black,
    resolution = (2000, 2000),
    whole_figure_title = "Leading EOF Modes of Monthly Averages of Surface Pressure (50 Winter Period)",
    shading=NoShading,
    surface_alpha_value=0.5
)

MethodError: MethodError: no method matching initialize_block!(::GeoAxis; transformation::@NamedTuple{translation::Tuple{Int64, Int64, Int64}})

Closest candidates are:
  initialize_block!(::GeoAxis) got unsupported keyword argument "transformation"
   @ GeoMakie ~/.julia/packages/GeoMakie/vcMuZ/src/geoaxis.jl:543
  initialize_block!(!Matched::Menu; default) got unsupported keyword argument "transformation"
   @ Makie ~/.julia/packages/Makie/ND0gA/src/makielayout/blocks/menu.jl:51
  initialize_block!(!Matched::Textbox) got unsupported keyword argument "transformation"
   @ Makie ~/.julia/packages/Makie/ND0gA/src/makielayout/blocks/textbox.jl:4
  ...


In [29]:
DataInspector(figure)

DataInspector(Scene (2000px, 1371px):
  1 Plot:
    └ Plot{Makie.tooltip, Tuple{Point{2, Float32}}}
  12 Child Scenes:
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    ├ Scene (2000px, 1371px)
    └ Scene (2000px, 1371px), Attributes with 11 entries:
  _color => RGBA{Float32}(0.0,0.0,0.0,0.0)
  apply_tooltip_offset => true
  depth => 9000.0
  enable_indicators => true
  enabled => true
  indicator_color => red
  indicator_linestyle => nothing
  indicator_linewidth => 2
  indicator_visible => false
  offset => 10.0
  range => 10, AbstractPlot[], Plot{Makie.tooltip, Tuple{Point{2, Float32}}}, Plot{Makie.tooltip, Tuple{Point{2, Float32}}}, Any[ObserverFunction defined at /home/denis/.julia/packages/Makie/ND0gA/src/interaction/inspector.jl:280 opera

In [30]:
figure